In [1]:
#모든 행과 열 표시
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
#최대 행과 열 표시 설정을 기본값으로 되돌리기
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')

In [1]:
import pandas as pd
import re

In [2]:
def specical_char_ratio(text):
    noise_count = len(re.findall(r"[^a-zA-Z0-9\sㄱ-ㅎㅏ-ㅣ가-힣一-龥]", text))
    return noise_count / len(text) if len(text) > 0 else 0

In [3]:
# 소문자 알파벳 비율
def alphabet_ratio(text):
    alphabet_count = len(re.findall(r'[a-z]', text))
    return alphabet_count / len(text) if len(text) > 0 else 0

In [4]:
# 대소문자 알파벳 비율
def all_alphabet_ratio(text):
    alphabet_count = len(re.findall(r'[A-Za-z]', text))
    return alphabet_count / len(text) if len(text) > 0 else 0

In [5]:
# 특수문자 제거
def remove_special_characters(text):
    return re.sub(r'[!#$\'\"\(\)*+\-/:;<=>?@\[\\\]^_`{|}＋&]+', '', text)

In [6]:
def save_dataframe_to_csv(dataframe, output_path, index=False):
    dataframe.to_csv(output_path, index=index)

In [7]:
df = pd.read_csv('data/train.csv')

### 기사 제목에 잘 사용하지 않는 특수문자를 제거한다.
[!#$\'\"\(\)*+\-/:;<=>?@\[\\\]^_`{|}＋&]+

In [8]:
# 특수문자 제거
df['remove_special_char'] = df['text'].apply(remove_special_characters)
df.head(5)

ID                              text  target  \
0  ynat-v1_train_00000  정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보       4   
1  ynat-v1_train_00001       K찰.국DLwo 로L3한N% 회장 2 T0&}송=       3   
2  ynat-v1_train_00002            m 김정) 자주통일 새,?r열1나가야1보       2   
3  ynat-v1_train_00003     갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩       5   
4  ynat-v1_train_00004      pI美대선I앞두고 R2fr단 발] $비해 감시 강화       6   

             remove_special_char  
0  정i 파1 미사z KT 이용기간 2e 단 Q분종U2보  
1       K찰.국DLwo 로L3한N% 회장 2 T0송  
2           m 김정 자주통일 새,r열1나가야1보  
3  갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩  
4     pI美대선I앞두고 R2fr단 발 비해 감시 강화

In [9]:
# text 앞뒤에 그래픽/게시판/종합 등이 나오는 경우 삭제(큰 의미 없음)
remove_special_char_df = df[['ID','remove_special_char','target']].rename(columns={'remove_special_char': 'text'})
remove_special_char_df['text'] = remove_special_char_df['text'].replace(r'^(그래픽|게시판)', '', regex=True)
remove_special_char_df['text'] = remove_special_char_df['text'].replace(r'^(종합|1보|2보)', '', regex=True)
remove_special_char_df['text'] = remove_special_char_df['text'].replace(r'(종합|종합1보|종합2보|1보|2보)$', '', regex=True)
remove_special_char_df.head(5)

ID                           text  target
0  ynat-v1_train_00000    정i 파1 미사z KT 이용기간 2e 단 Q분종U       4
1  ynat-v1_train_00001       K찰.국DLwo 로L3한N% 회장 2 T0송       3
2  ynat-v1_train_00002             m 김정 자주통일 새,r열1나가야       2
3  ynat-v1_train_00003  갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩       5
4  ynat-v1_train_00004     pI美대선I앞두고 R2fr단 발 비해 감시 강화       6

## 특수문자가 하나도 제거되지 않은 경우
노이즈가 추가되지 않았을 확률이 높은 데이터라고 가정한다.

In [10]:
# text와 특수문자제거 text의 길이 차이를 확인
df['text_length'] = df['text'].apply(len)
df['remove_special_char_length'] = df['remove_special_char'].apply(len)
df['length_diff'] = df['text_length'] - df['remove_special_char_length']

In [11]:
# 특수문자가 제거되지 않은 경우
length_diff_0 = df[df['length_diff'] == 0]
length_diff_0 = length_diff_0[['ID','text','target','length_diff']]
length_diff_0['text'] = length_diff_0['text'].replace(r'^(그래픽|게시판)', '', regex=True)
length_diff_0['text'] = length_diff_0['text'].replace(r'^(종합|1보|2보)', '', regex=True)
length_diff_0['text'] = length_diff_0['text'].replace(r'(종합|종합1보|종합2보|1보|2보)$', '', regex=True)
length_diff_0.head(5)

ID                              text  target  length_diff
3  ynat-v1_train_00003     갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩       5            0
5  ynat-v1_train_00005    美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다       0            0
6  ynat-v1_train_00006             프로야구~롯TKIAs광주 경기 y천취소       1            0
7  ynat-v1_train_00007  아가메즈 33득점 우리카드 KB손해보험 완파…3위 굳...       4            0
8  ynat-v1_train_00008      朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문       6            0

## 노이즈가 추가되지 않았을 확률이 높은 데이터 중 소문자의 비율이 0이 아닌 경우
이를 노이즈가 추가된 데이터라고 가정한다. 그 이유는, 대부분의 뉴스 기사의 영어는 대문자로 작성하기 때문이다. 단, 예외가 존재하는데 예를 들어 IoT, 단위를 나타내는 m 등은 걸러낼 수 없다.

In [12]:
# 특수문자가 제거되지 않은 경우 소문자의 비율을 확인한다.
length_diff_0['alphabet_ratio'] = length_diff_0['text'].apply(alphabet_ratio)

no noise data 특수문자가 한개도 제거되지 않고 소문자를 한개도 포함하지 않은 경우

In [13]:
sorted_length_diff_0= length_diff_0.sort_values(by='alphabet_ratio', ascending=False)
no_noise_df = sorted_length_diff_0[sorted_length_diff_0['alphabet_ratio']== 0.0]
no_noise_df.head(5)

ID                            text  target  \
1893  ynat-v1_train_01893     다승 1위 두산 이용찬 옆구리 통증으로 1군 말소       0   
1880  ynat-v1_train_01880        대전협 비대위 의결 과정에 절차상 문제없었다       1   
1836  ynat-v1_train_01836    프로야구 SK 신인 상시 교육…새로운 팀 문화 기대       3   
1883  ynat-v1_train_01883         옛 인기예능 모아보자…KBS 깔깔티비 오픈       6   
1885  ynat-v1_train_01885  하루키 한정판·애니 너의 이름은.…새해 日소설 인기몰이       2   

      length_diff  alphabet_ratio  
1893            0             0.0  
1880            0             0.0  
1836            0             0.0  
1883            0             0.0  
1885            0             0.0

noise data1
특수문자가 1개 이상 제거된 경우

In [14]:
noise_data1 = df[df['length_diff'] != 0]
noise_data1 = noise_data1[['ID','text','target']]
noise_data1.head(5)

ID                              text  target
0   ynat-v1_train_00000  정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보       4
1   ynat-v1_train_00001       K찰.국DLwo 로L3한N% 회장 2 T0&}송=       3
2   ynat-v1_train_00002            m 김정) 자주통일 새,?r열1나가야1보       2
4   ynat-v1_train_00004      pI美대선I앞두고 R2fr단 발] $비해 감시 강화       6
10  ynat-v1_train_00010       oi 매력 R모h츠a열#w3약 >l·주가 고Q/진       5

noise data2 특수문자가 하나도 제거되지 않았지만, 제목에 소문자가 포함 되어있는 경우

In [15]:
noise_data2 = sorted_length_diff_0[sorted_length_diff_0['alphabet_ratio']!= 0.0]
noise_data2 = noise_data2[['ID','text','target']]
noise_data2

ID                                  text  target
2037  ynat-v1_train_02037                       b저임tq만원 n약 p행Wh       3
247   ynat-v1_train_00247                       m시판 코.아 Vvg페ogt       4
1247  ynat-v1_train_01247         1폭a M,p넥wE박2원q조상qj25일 x찰 FwH사       3
636   ynat-v1_train_00636                       m기에 경례하5 이mt홍영c       2
586   ynat-v1_train_00586                   오Gc r권,bA채Xa리 소o 반등       5
...                   ...                                   ...     ...
654   ynat-v1_train_00654        S렘린궁 김정은 J원2UhGF하반기 러시아M방문H합2A       2
657   ynat-v1_train_00657      팀 쿡 애플 CEO iOS 11 배포 통해 증강현실 본격화       0
957   ynat-v1_train_00957      MWC19 SK텔레콤 싱가포르 통신사 싱텔과 e스포츠 협력       4
1784  ynat-v1_train_01784  영상 인니 경찰 2m 넘는 구렁이로 위협 심문…거짓말 하면 물린다       2
1987  ynat-v1_train_01987  G창 k천K 진입도로 .방도 승격…국P 50V억 지원 4차로 8장       3

[105 rows x 3 columns]

In [16]:
noise_df = pd.concat([noise_data1, noise_data2])
noise_df

ID                                  text  target
0     ynat-v1_train_00000      정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보       4
1     ynat-v1_train_00001           K찰.국DLwo 로L3한N% 회장 2 T0&}송=       3
2     ynat-v1_train_00002                m 김정) 자주통일 새,?r열1나가야1보       2
4     ynat-v1_train_00004          pI美대선I앞두고 R2fr단 발] $비해 감시 강화       6
10    ynat-v1_train_00010           oi 매력 R모h츠a열#w3약 >l·주가 고Q/진       5
...                   ...                                   ...     ...
654   ynat-v1_train_00654        S렘린궁 김정은 J원2UhGF하반기 러시아M방문H합2A       2
657   ynat-v1_train_00657      팀 쿡 애플 CEO iOS 11 배포 통해 증강현실 본격화       0
957   ynat-v1_train_00957      MWC19 SK텔레콤 싱가포르 통신사 싱텔과 e스포츠 협력       4
1784  ynat-v1_train_01784  영상 인니 경찰 2m 넘는 구렁이로 위협 심문…거짓말 하면 물린다       2
1987  ynat-v1_train_01987  G창 k천K 진입도로 .방도 승격…국P 50V억 지원 4차로 8장       3

[1611 rows x 3 columns]

In [17]:
no_noise_df['is_noise'] = 0
noise_df['is_noise'] = 1
split_noise_nonoise = pd.concat([no_noise_df, noise_df])
split_noise_nonoise

/var/folders/kd/_nb0bhb56nv8z6yw94zdh64c0000gn/T/ipykernel_3458/603377872.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_noise_df['is_noise'] = 0


ID                                  text  target  \
1893  ynat-v1_train_01893           다승 1위 두산 이용찬 옆구리 통증으로 1군 말소       0   
1880  ynat-v1_train_01880              대전협 비대위 의결 과정에 절차상 문제없었다       1   
1836  ynat-v1_train_01836          프로야구 SK 신인 상시 교육…새로운 팀 문화 기대       3   
1883  ynat-v1_train_01883               옛 인기예능 모아보자…KBS 깔깔티비 오픈       6   
1885  ynat-v1_train_01885        하루키 한정판·애니 너의 이름은.…새해 日소설 인기몰이       2   
...                   ...                                   ...     ...   
654   ynat-v1_train_00654        S렘린궁 김정은 J원2UhGF하반기 러시아M방문H합2A       2   
657   ynat-v1_train_00657      팀 쿡 애플 CEO iOS 11 배포 통해 증강현실 본격화       0   
957   ynat-v1_train_00957      MWC19 SK텔레콤 싱가포르 통신사 싱텔과 e스포츠 협력       4   
1784  ynat-v1_train_01784  영상 인니 경찰 2m 넘는 구렁이로 위협 심문…거짓말 하면 물린다       2   
1987  ynat-v1_train_01987  G창 k천K 진입도로 .방도 승격…국P 50V억 지원 4차로 8장       3   

      length_diff  alphabet_ratio  is_noise  
1893          0.0             0.0         0  
1880          0.0             0.0         0  
1836          0.0             0.0         0  
1883          0.0             0.0         0  
1885          0.0             0.0         0  
...           ...             ...       ...  
654           NaN             NaN         1  
657           NaN             NaN         1  
957           NaN             NaN         1  
1784          NaN             NaN         1  
1987          NaN             NaN         1  

[2800 rows x 6 columns]

In [18]:
split_noise_nonoise = split_noise_nonoise[['ID','text','target','is_noise']]
split_noise_nonoise

ID                                  text  target  \
1893  ynat-v1_train_01893           다승 1위 두산 이용찬 옆구리 통증으로 1군 말소       0   
1880  ynat-v1_train_01880              대전협 비대위 의결 과정에 절차상 문제없었다       1   
1836  ynat-v1_train_01836          프로야구 SK 신인 상시 교육…새로운 팀 문화 기대       3   
1883  ynat-v1_train_01883               옛 인기예능 모아보자…KBS 깔깔티비 오픈       6   
1885  ynat-v1_train_01885        하루키 한정판·애니 너의 이름은.…새해 日소설 인기몰이       2   
...                   ...                                   ...     ...   
654   ynat-v1_train_00654        S렘린궁 김정은 J원2UhGF하반기 러시아M방문H합2A       2   
657   ynat-v1_train_00657      팀 쿡 애플 CEO iOS 11 배포 통해 증강현실 본격화       0   
957   ynat-v1_train_00957      MWC19 SK텔레콤 싱가포르 통신사 싱텔과 e스포츠 협력       4   
1784  ynat-v1_train_01784  영상 인니 경찰 2m 넘는 구렁이로 위협 심문…거짓말 하면 물린다       2   
1987  ynat-v1_train_01987  G창 k천K 진입도로 .방도 승격…국P 50V억 지원 4차로 8장       3   

      is_noise  
1893         0  
1880         0  
1836         0  
1883         0  
1885         0  
...        ...  
654          1  
657          1  
957          1  
1784         1  
1987         1  

[2800 rows x 4 columns]

In [19]:
split_noise_nonoise.to_csv('data/1_noise_detected.csv', index=False)